In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import age_cal as ag
import cv2
import dlib
import imutils
import os

In [ ]:
!pip install matplotlib

In [3]:
df = pd.read_csv("./dataset/csv/real_person.csv")

new_df = df[["name","date_of_birth","weight","sex","height"]]

initial_len = len(new_df)

new_df.dropna(inplace=True)

final_len = len(new_df)

print(f"No of data removed:{initial_len - final_len}")

new_df

No of data removed:397


C:\Users\veere\AppData\Local\Temp\ipykernel_18824\3580997671.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)


,name,date_of_birth,weight,sex,height
A00147,JOHN,06/14/1949,185.0,Male,67.0
A00220,ISIAH,03/30/1957,155.0,Male,73.0
A00360,HOWARD,12/18/1946,167.0,Male,69.0
A00367,RAYMOND,01/12/1954,245.0,Male,72.0
A01054,DARNELL,03/25/1954,166.0,Male,67.0
...,...,...,...,...,...
Y25362,ERIC A.,05/30/1985,120.0,Male,59.0
Y25363,JEFFERY D.,05/15/1986,170.0,Male,71.0
Y25364,KIMBERLY D.,02/23/1972,112.0,Female,62.0
Y25365,KARLEE R.,06/16/1992,158.0,Female,63.0


In [5]:
import importlib

importlib.reload(ag)

new_df['Age'] = new_df['date_of_birth'].apply(ag.calculate_age)

new_df['height'] = new_df['height'] * 0.0254  # converts height to meters
new_df['weight'] = new_df['weight'] * 0.4536  # converts weight to 

new_df['BMI'] = new_df.apply(lambda row: ag.calculate_bmi(row['height'], row['weight']), axis=1)

new_df

C:\Users\veere\AppData\Local\Temp\ipykernel_18824\4262645238.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Age'] = new_df['date_of_birth'].apply(ag.calculate_age)
C:\Users\veere\AppData\Local\Temp\ipykernel_18824\4262645238.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['height'] = new_df['height'] * 0.0254  # converts height to meters
C:\Users\veere\AppData\Local\Temp\ipykernel_18824\4262645238.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,name,date_of_birth,weight,sex,height,Age,BMI
A00147,JOHN,06/14/1949,83.9160,Male,1.7018,75,28.975286
A00220,ISIAH,03/30/1957,70.3080,Male,1.8542,67,20.449919
A00360,HOWARD,12/18/1946,75.7512,Male,1.7526,77,24.661751
A00367,RAYMOND,01/12/1954,111.1320,Male,1.8288,70,33.228191
A01054,DARNELL,03/25/1954,75.2976,Male,1.7018,70,25.999446
...,...,...,...,...,...,...,...
Y25362,ERIC A.,05/30/1985,54.4320,Male,1.4986,39,24.237222
Y25363,JEFFERY D.,05/15/1986,77.1120,Male,1.8034,38,23.710343
Y25364,KIMBERLY D.,02/23/1972,50.8032,Female,1.5748,52,20.485202
Y25365,KARLEE R.,06/16/1992,71.6688,Female,1.6002,32,27.988627


In [7]:
#extracting facial features from the image using a pretrained model
# Load the pre-trained facial landmarks model
landmark_model_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(landmark_model_path)

def extract_landmarks(image_path):
    # Read the image
    image = cv2.imread(image_path)
    image = imutils.resize(image, width=600)

    if image is None:
        print("Hello")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = detector(gray)
    if len(faces) == 0:
        print("No faces detected!")
        return None

    # Assume the first face detected is the target face
    landmarks = predictor(gray, faces[0])

    # Extract (x, y) coordinates of the landmarks
    points = []
    for n in range(68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        points.append((x, y))

    return points

def compute_features(front_landmarks, side_landmarks=None):
    # Cheek-to-Jaw Width Ratio (CJWR)
    jaw_width = np.linalg.norm(np.array(front_landmarks[3]) - np.array(front_landmarks[13]))
    cheek_width = np.linalg.norm(np.array(front_landmarks[1]) - np.array(front_landmarks[15]))
    cjwr = cheek_width / jaw_width

    # Width to Upper Facial Height Ratio (WHR)
    width = np.linalg.norm(np.array(front_landmarks[1]) - np.array(front_landmarks[15]))
    upper_height = np.linalg.norm(np.array(front_landmarks[27]) - np.array(front_landmarks[8]))
    whr = width / upper_height

    # Perimeter to Area Ratio (PAR)
    contour_points = np.array(front_landmarks[:17])  # Jawline points
    perimeter = cv2.arcLength(contour_points, True)
    area = cv2.contourArea(contour_points)
    par = perimeter / area if area != 0 else None

    # Eye Size (ES)
    left_eye_width = np.linalg.norm(np.array(front_landmarks[36]) - np.array(front_landmarks[39]))
    left_eye_height = np.linalg.norm(np.array(front_landmarks[37]) - np.array(front_landmarks[41]))
    right_eye_width = np.linalg.norm(np.array(front_landmarks[42]) - np.array(front_landmarks[45]))
    right_eye_height = np.linalg.norm(np.array(front_landmarks[43]) - np.array(front_landmarks[47]))
    es = ((left_eye_width * left_eye_height) + (right_eye_width * right_eye_height)) / 2

    # Lower Face to Face Height Ratio (FW/FH)
    lower_face_height = np.linalg.norm(np.array(front_landmarks[33]) - np.array(front_landmarks[8]))
    face_height = np.linalg.norm(np.array(front_landmarks[27]) - np.array(front_landmarks[8]))
    fw_fh = lower_face_height / face_height

    # Mean Eyebrow Height (MEH)
    left_eyebrow_mean_height = np.mean([np.linalg.norm(np.array(front_landmarks[19]) - np.array(front_landmarks[37])),
                                        np.linalg.norm(np.array(front_landmarks[20]) - np.array(front_landmarks[38]))])
    right_eyebrow_mean_height = np.mean([np.linalg.norm(np.array(front_landmarks[23]) - np.array(front_landmarks[43])),
                                         np.linalg.norm(np.array(front_landmarks[24]) - np.array(front_landmarks[44]))])
    meh = (left_eyebrow_mean_height + right_eyebrow_mean_height) / 2

    return {
        "CJWR": cjwr,
        "WHR": whr,
        "PAR": par,
        "ES": es,
        "FW/FH": fw_fh,
        "MEH": meh
    }

# Example usage
front_photo_path = "A00147_front.jpg"
side_photo_path = "A00147_side.jpg"  # Optional for advanced features

front_landmarks = extract_landmarks(front_photo_path)
if front_landmarks:
    features = compute_features(front_landmarks)
    print("Computed Facial Features:")
    for key, value in features.items():
        print(f"{key}: {value}") 


Computed Facial Features:
CJWR: 1.081183738441964
WHR: 1.0675000749332786
PAR: 0.018273272652999176
ES: 411.32839005057497
FW/FH: 0.6458625098059632
MEH: 32.65681192401898


In [17]:
# compute the frontal and side wise facial features for each entry and add this to the dataframe

cols = ['front_CJWR', 'front_WHR', 'front_PAR', 'front_ES', 'front_FW/FH', 'front_MEH', 'side_CJWR', 'side_WHR', 'side_PAR', 'side_ES', 'side_FW/FH', 'side_MEH']

for c in cols:
    new_df[c] = 0

index_values = new_df.index.tolist()

path = './dataset/front/front/'

new_df

# for i in index_values:
#     file_path = path + i + ".jpg"

#     if(os.exists(file_path)):
        
        




C:\Users\veere\AppData\Local\Temp\ipykernel_18824\2658455622.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[c] = 0


,name,date_of_birth,weight,sex,height,Age,BMI,front_CJWR,front_WHR,front_PAR,front_ES,front_FW/FH,front_MEH,side_CJWR,side_WHR,side_PAR,side_ES,side_FW/FH,side_MEH
A00147,JOHN,06/14/1949,83.9160,Male,1.7018,75,28.975286,0,0,0,0,0,0,0,0,0,0,0,0
A00220,ISIAH,03/30/1957,70.3080,Male,1.8542,67,20.449919,0,0,0,0,0,0,0,0,0,0,0,0
A00360,HOWARD,12/18/1946,75.7512,Male,1.7526,77,24.661751,0,0,0,0,0,0,0,0,0,0,0,0
A00367,RAYMOND,01/12/1954,111.1320,Male,1.8288,70,33.228191,0,0,0,0,0,0,0,0,0,0,0,0
A01054,DARNELL,03/25/1954,75.2976,Male,1.7018,70,25.999446,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Y25362,ERIC A.,05/30/1985,54.4320,Male,1.4986,39,24.237222,0,0,0,0,0,0,0,0,0,0,0,0
Y25363,JEFFERY D.,05/15/1986,77.1120,Male,1.8034,38,23.710343,0,0,0,0,0,0,0,0,0,0,0,0
Y25364,KIMBERLY D.,02/23/1972,50.8032,Female,1.5748,52,20.485202,0,0,0,0,0,0,0,0,0,0,0,0
Y25365,KARLEE R.,06/16/1992,71.6688,Female,1.6002,32,27.988627,0,0,0,0,0,0,0,0,0,0,0,0
